https://blog.quantinsti.com/heston-model/

# Heston 1993 Model

## Vanilla Option 

In [13]:
import numpy as np

def heston_charfunc(u, params):
    S0   = params["S0"]
    K    = params["K"]
    r    = params["r"]
    q    = params["q"]
    v0   = params["v0"]
    kappa= params["kappa"]
    theta= params["theta"]
    sigma= params["sigma"]
    rho  = params["rho"]
    T    = params["T"]

    iu = 1j * u

    alpha = -0.5 * (u**2 + iu)
    beta  = kappa - rho * sigma * iu
    gamma = 0.5 * sigma**2

    d = np.sqrt(beta**2 - 4.0 * alpha * gamma)
    g = (beta - d) / (beta + d)

    exp_dT = np.exp(-d * T)

    C = (r - q) * iu * T + \
        (kappa * theta / gamma) * ((beta - d) * T - 2.0 * np.log((1.0 - g * exp_dT) / (1.0 - g)))

    D = (beta - d) / gamma * ((1.0 - exp_dT) / (1.0 - g * exp_dT))

    return np.exp(C + D * v0 + iu * np.log(S0 * np.exp(-q * T)))


def heston_price_call_fft(params, N=10000, U_max=1000):

    S0 = params["S0"]
    r  = params["r"]
    q  = params["q"]
    T  = params["T"]
    K  = params["K"]

    if N % 2 == 1:
        N += 1

    u = np.linspace(1e-10, U_max, N + 1)
    du = u[1] - u[0]
    lnK = np.log(K)

    phi_u     = heston_charfunc(u, params) #Parties Utilisées dans l'intégration
    phi_u_im1 = heston_charfunc(u - 1j, params)
    phi_im1   = heston_charfunc(-1j, params)

    integrand_P1 = np.real(np.exp(-1j * u * lnK) * phi_u_im1 /
                           (1j * u * phi_im1)) #Partie réelle de l'intégrale
    integrand_P2 = np.real(np.exp(-1j * u * lnK) * phi_u /
                           (1j * u)) #Partie réelle de l'intégrale

    weights = np.ones(N + 1)
    weights[1:-1:2] = 4.0 #Poids Simpson impairs
    weights[2:-2:2] = 2.0 #Poids Simpson pairs

    P1 = 0.5 + (du / (3.0 * np.pi)) * np.sum(weights * integrand_P1) #Ponderation Simpson
    P2 = 0.5 + (du / (3.0 * np.pi)) * np.sum(weights * integrand_P2)

    call = S0 * np.exp(-q * T) * P1 - K * np.exp(-r * T) * P2
    put = K*np.exp(-r*T)*(1-P2) - S0*np.exp(-q*T)*(1-P1)
    return call, put

In [14]:
# Example
params = {
    "S0": 100,
    "K" : 105,
    "r":  0.02,
    "q": 0.01,
    "v0": 0.08,
    "kappa": 1.12,
    "theta": 0.05,
    "sigma": 0.50,
    "rho": -0.25,
    "T": 0.10
}

call, put = heston_price_call_fft(params)
print(f"Call Price: {call:.4f}")
print(f"Put Price: {put:.4f}")

Call Price: 2.9539
Put Price: 7.8441


## Extract Data before calibrating the model

In [15]:
#import pandas as pd

# 1) On définit d’abord les maturités (index) et les niveaux de strikes (colonnes)
#maturities = ['T', '1M', '2M', '3M', '6M', '9M', '1Y', '18M', '2Y', '3Y', '4Y', '5Y', '7Y', '10Y']
#strikes_pct = ['80%', '90%', '95%', '97.5%', '100%', '102.5%', '105%', '110%', '120%']

# 2) On crée ensuite la liste des listes « raw » correspondant aux lignes du tableau
#data = [
    # Strike prices (ligne 'T')
#    [3036.59, 3416.17, 3605.95, 3700.85, 3795.74, 3890.63, 3985.53, 4175.31, 4554.89],
    # Valeurs pour 1M
#    [25.29,   19.15,   15.35,   13.70,   12.25,   11.04,   10.71,   11.46,   13.56],
    # 2M
#    [24.28,   18.46,   15.81,   14.58,   13.43,   12.39,   11.67,   11.52,   12.55],
    # 3M
#    [23.03,   18.08,   15.96,   14.96,   14.00,   13.10,   12.38,   11.83,   12.06],
    # 6M
#    [21.93,   18.24,   16.61,   15.85,   15.13,   14.47,   13.86,   12.87,   12.40],
    # 9M
#    [21.22,   18.35,   17.05,   16.41,   15.79,   15.20,   14.65,   13.71,   12.72],
    # 1Y
#    [21.14,   18.61,   17.48,   16.93,   16.39,   15.86,   15.35,   14.43,   13.28],
    # 18M
#    [21.15,   19.15,   18.18,   17.71,   17.24,   16.79,   16.36,   15.58,   14.39],
    # 2Y
#    [21.03,   19.33,   18.51,   18.10,   17.71,   17.33,   16.97,   16.30,   15.22],
    # 3Y
#    [21.19,   19.68,   18.95,   18.59,   18.25,   17.91,   17.59,   16.99,   15.98],
    # 4Y
#    [21.29,   19.99,   19.36,   19.06,   18.77,   18.49,   18.22,   17.72,   16.84],
    # 5Y
#    [21.52,   20.36,   19.82,   19.56,   19.31,   19.06,   18.82,   18.38,   17.60],
    # 7Y
#    [22.14,   21.18,   20.74,   20.53,   20.33,   20.13,   19.94,   19.58,   18.93],
    # 10Y
#    [23.17,   22.41,   22.06,   21.89,   21.73,   21.57,   21.42,   21.14,   20.62],
#]

# 3) On construit le DataFrame
#df = pd.DataFrame(data, index=maturities, columns=strikes_pct)

# 4) Optionnel : convertir en float (ici déjà float)
#df = df.astype(float)

#df.head()


In [16]:
import pandas as pd

excel_file = "market_data.xlsx"

wb = pd.read_excel(excel_file,sheet_name='SPX-2014' ,header=None,)


header_row_idx = 6
start_col = 2
end_col = 13               
n_maturities = 14

raw_header = wb.iloc[header_row_idx, start_col:end_col].tolist()
raw_data   = wb.iloc[
    header_row_idx+1 : header_row_idx+1+n_maturities,
    start_col : end_col
]


df = pd.DataFrame(
    raw_data.values[:, 1:],          
    index=raw_data.values[:, 0],    
    columns=raw_header[1:],         
)


df.columns = [f"{x:.1%}" for x in df.columns.astype(float)]


print(df.head())

S0 = wb.iat[3, 4]

r = wb.iat[1, 10]

q = wb.iat[2, 10]

       80.0%     90.0%     95.0%     97.5%   100.0%    102.5%    105.0%  \
T   1560.928  1756.044  1853.602  1902.381  1951.16  1999.939  2048.718   
1M     26.98      18.2     13.37     11.19     9.22      8.03      8.12   
2M      23.4     16.63     13.64      12.1    10.61       9.3      8.66   
3M     21.94     16.51     13.87     12.53    11.24     10.09       9.3   
6M     20.71     16.72     14.68     13.67    12.71     11.79     10.97   

      110.0%    120.0%  
T   2146.276  2341.392  
1M      8.35     10.35  
2M      8.63       9.6  
3M      8.83      9.31  
6M     10.02      9.82  


## Calibration

In [17]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares
from scipy.stats import norm

# Black–Scholes call price pour reconstruire les prix marchés à partir des vols implicites

def bs_call_price(S, K, r, q, sigma, T):
    d1 = (np.log(S/K) + (r - q + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*np.exp(-q*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)


# Conversion des labels de maturités en années
def maturity_to_T(label):
    if label == 'T':
        return None
    if label.endswith('M'):
        return int(label[:-1]) / 12.0
    if label.endswith('Y'):
        return float(label[:-1])
    raise ValueError(label)

# Construction d’une liste de points marché (T, K, prix_call)
market_data = []
strikes_abs = df.loc['T']  
for mat in df.index:
    T = maturity_to_T(mat)
    if T is None:
        continue
    for pct in df.columns:
        K      = strikes_abs[pct]
        vol_imp= df.loc[mat, pct]/100.0  # en pourcentage
        price  = bs_call_price(S0, K, r, q, vol_imp, T)
        market_data.append((T, K, price))

market_data = np.array(market_data, 
                       dtype=[('T', float), ('K', float), ('price', float)]) #Surface de prix


#3) Fonction objectif pour l’optimiseur 

def pack_params(x):
    """Dépaquète le vecteur x en dictionnaire de paramètres Heston."""
    return {
        "S0":   S0,
        "r":    r,
        "q":    q,
        "v0":   x[0],
        "kappa":x[1],
        "theta":x[2],
        "sigma":x[3],
        "rho":  x[4],
    }

def residuals(x):
    """Retourne les écarts modèle – marché pour chaque point de la surface."""
    params_base = pack_params(x)
    res = []
    for T, K, mkt_price in market_data:
        params = params_base.copy()
        params.update({"T": T, "K": K})
        model_call, _ = heston_price_call_fft(params, N=1000, U_max=2000)
        res.append(model_call - mkt_price)
    return np.array(res)


#4) Calibration par moindres carrés non‑linéaires 

# 4.1) Initial guess and bounds
x0 = np.array([
    0.04,    # v0
    1.0,     # kappa
    0.04,    # theta
    0.5,     # sigma
    -0.5,    # rho
])

# on impose rho ∈ [−0.999, 0.999] et les autres ≥ 0
lb = [1e-4, 1e-4, 1e-4, 1e-4, -0.999]
ub = [2.0, 10.0, 2.0, 5.0, 0.999]

opt = least_squares(
    residuals,
    x0,
    bounds=(lb, ub),
    verbose=2,
    xtol=1e-6,
    ftol=1e-6,
    max_nfev=200
)

# 4.2) Résultats
v0_fit, kappa_fit, theta_fit, sigma_fit, rho_fit = opt.x
print("Paramètres calibrés Heston :")
print(f" v0    = {v0_fit:.4f}")
print(f" kappa = {kappa_fit:.4f}")
print(f" theta = {theta_fit:.4f}")
print(f" sigma = {sigma_fit:.4f}")
print(f" rho   = {rho_fit:.4f}")


   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         2.8904e+05                                    2.39e+06    
       1              2         3.1241e+04      2.58e+05       5.78e-01       2.22e+05    
       2              3         7.0250e+03      2.42e+04       4.77e-01       4.88e+04    
       3              4         3.0796e+03      3.95e+03       1.11e-01       4.25e+03    
       4              5         1.4028e+03      1.68e+03       8.23e-01       3.66e+04    
       5              6         1.1181e+03      2.85e+02       1.20e-01       7.79e+04    
       6              8         9.5213e+02      1.66e+02       1.20e-01       3.32e+05    
       7              9         8.1968e+02      1.32e+02       1.70e-01       7.08e+05    
       8             10         6.5529e+02      1.64e+02       5.29e-02       1.60e+05    
       9             11         6.1359e+02      4.17e+01       9.41e-02       3.64e+05    

In [18]:
# Test de la calibration
# On compare les prix Heston et Black-Scholes
print("Prix Black-Sholes, M=1Y, K=100%:", bs_call_price(S0,S0,r,q,0.1421,1.0))
params = {
    "S0": S0,
    "K" : S0,
    "r":  r ,
    "q": q ,
    "v0": v0_fit,
    "kappa": kappa_fit,
    "theta": theta_fit,
    "sigma": sigma_fit,
    "rho": rho_fit,
    "T": 1.0
}

print("Prix Heston, M=1Y, K=100%:", heston_price_call_fft(params, N=1000, U_max=2000)[0])

Prix Black-Sholes, M=1Y, K=100%: 100.47506469207599
Prix Heston, M=1Y, K=100%: 95.9165630162181


# Binary Option 

In [19]:
def heston_price_binary_fft(params, N=10000, U_max=1000):
    """
    Calculate binary option prices using the Heston model with FFT
    
    Parameters:
    params (dict): Model parameters
    N (int): Number of integration points
    U_max (float): Upper integration limit
    
    Returns:
    float: Binary option price
    """
    S0 = params["S0"]
    r  = params["r"]
    q  = params["q"]
    T  = params["T"]
    K  = params["K"]

    if N % 2 == 1:
        N += 1

    u = np.linspace(1e-10, U_max, N + 1)
    du = u[1] - u[0]
    lnK = np.log(K)

    # Calculate characteristic function
    phi_u = heston_charfunc(u, params)

    # For binary options, we only need P2
    integrand = np.real(np.exp(-1j * u * lnK) * phi_u / (1j * u))

    # Simpson's weights
    weights = np.ones(N + 1)
    weights[1:-1:2] = 4.0
    weights[2:-2:2] = 2.0

    # Probability calculation
    P2 = 0.5 + (du / (3.0 * np.pi)) * np.sum(weights * integrand)

    # Binary option payoffs
    binary_call = np.exp(-r * T) * P2
    binary_put = np.exp(-r * T) * (1 - P2)

    return binary_call, binary_put

    

In [20]:
binary_call,binary_put = heston_price_binary_fft(params)
print(f"Binary Call Price: {binary_call:.4f}")
print(f"Binary Put Price: {binary_put:.4f}")

Binary Call Price: 0.4500
Binary Put Price: 0.5361


# Variance Swaps

In [21]:
def heston_variance_swap_strike(params):
    """
    Calculate the fair variance swap strike under Heston model
    
    Parameters:
    params (dict): Model parameters including v0, kappa, theta, T
    
    Returns:
    float: Fair variance swap strike (in variance terms)
    """
    v0 = params["v0"]
    kappa = params["kappa"]
    theta = params["theta"]
    T = params["T"]
    
    # The fair strike is the expected average variance under Q
    # E_Q[1/T ∫_0^T v_t dt] = theta + (v0 - theta)*(1 - exp(-kappa*T))/(kappa*T)
    fair_variance = theta + (v0 - theta) * (1 - np.exp(-kappa * T)) / (kappa * T)
    
    return fair_variance

# Example usage with your calibrated parameters
params_var_swap = {
    "v0": v0_fit,
    "kappa": kappa_fit,
    "theta": theta_fit,
    "T": 1.0  # 1-year variance swap
}

variance_strike = heston_variance_swap_strike(params_var_swap)
volatility_strike = np.sqrt(variance_strike) * 100  # Convert to volatility in percentage

print(f"Variance Swap Strike (variance): {variance_strike:.6f}")
print(f"Variance Swap Strike (vol %)   : {volatility_strike:.2f}%")

Variance Swap Strike (variance): 0.013555
Variance Swap Strike (vol %)   : 11.64%
